In [2]:
def crc_remainder(data, poly):
    # Προσθήκη μηδενικών
    data = data + '0' * (len(poly) - 1)
    data = list(map(int, data))
    poly = list(map(int, poly))

    for i in range(len(data) - len(poly) + 1):
        if data[i] == 1:
            for j in range(len(poly)):
                data[i + j] ^= poly[j]

    return ''.join(map(str, data[-(len(poly)-1):]))

# Εκτέλεση
message = "101011010111"
polynomial = "1010011"
crc = crc_remainder(message, polynomial)
print("CRC:", crc)

CRC: 111100


In [3]:
# ================================================================
#implementation of CRC, Hamming, Dijkstra, Bellman-Ford
# ================================================================

import sys
from collections import defaultdict, deque
import heapq

# ------------------------------
# 1. CRC (Cyclic Redundancy Check)
# ------------------------------
def crc_remainder(data: str, generator: str) -> str:
    """Compute CRC remainder using modulo-2 division."""
    data = list(map(int, data))
    gen = list(map(int, generator))
    n = len(gen)
    # Append n-1 zeros
    for _ in range(n - 1):
        data.append(0)
    # Perform division
    for i in range(len(data) - n + 1):
        if data[i] == 1:
            for j in range(n):
                data[i + j] ^= gen[j]
    return ''.join(map(str, data[-(n - 1):]))


def crc_check(data_with_crc: str, generator: str) -> bool:
    """Verify CRC: remainder must be zero."""
    remainder = crc_remainder(data_with_crc, generator)
    return all(b == '0' for b in remainder)


# ------------------------------
# 2. Hamming Code (7,4) - Even Parity
# ------------------------------
def hamming_encode(data: str) -> str:
    """Encode 4-bit data into 7-bit Hamming code (even parity)."""
    if len(data) != 4:
        raise ValueError("Input must be 4 bits")
    d = [int(b) for b in data]
    p1 = d[0] ^ d[1] ^ d[3]
    p2 = d[0] ^ d[2] ^ d[3]
    p4 = d[1] ^ d[2] ^ d[3]
    return f"{p1}{p2}{d[0]}{p4}{d[1]}{d[2]}{d[3]}"


def hamming_decode(code: str) -> tuple[str, bool, int | None]:
    """Decode 7-bit Hamming code; returns (data, is_valid, error_position)"""
    if len(code) != 7:
        raise ValueError("Input must be 7 bits")
    c = [int(b) for b in code]
    # Parity checks
    p1 = c[0] ^ c[2] ^ c[4] ^ c[6]
    p2 = c[1] ^ c[2] ^ c[5] ^ c[6]
    p4 = c[3] ^ c[4] ^ c[5] ^ c[6]
    syndrome = p1 + (p2 << 1) + (p4 << 2)
    if syndrome == 0:
        return f"{c[2]}{c[4]}{c[5]}{c[6]}", True, None
    else:
        # Correct single-bit error
        c[syndrome - 1] ^= 1
        return f"{c[2]}{c[4]}{c[5]}{c[6]}", False, syndrome


# ------------------------------
# 3. Dijkstra's Algorithm
# ------------------------------
def dijkstra(graph: dict, start: str) -> dict:
    """Return shortest paths from start node."""
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    pq = [(0, start)]
    visited = set()
    while pq:
        current_dist, current = heapq.heappop(pq)
        if current in visited:
            continue
        visited.add(current)
        for neighbor, weight in graph[current].items():
            distance = current_dist + weight
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(pq, (distance, neighbor))
    return distances


# ------------------------------
# 4. Bellman-Ford Algorithm
# ------------------------------
def bellman_ford(edges: list[tuple], start: str, nodes: list[str]) -> dict | None:
    """Return shortest paths or None if negative cycle exists."""
    dist = {node: float('inf') for node in nodes}
    dist[start] = 0
    for _ in range(len(nodes) - 1):
        for u, v, w in edges:
            if dist[u] + w < dist[v]:
                dist[v] = dist[u] + w
    # Check for negative cycles
    for u, v, w in edges:
        if dist[u] + w < dist[v]:
            return None  # Negative cycle detected
    return dist


# ------------------------------
# Demo / Test Functions
# ------------------------------
if __name__ == "__main__":
    print("=== CRC Example ===")
    msg = "101011010111"
    poly = "1010011"  # x^6 + x^4 + x + 1
    crc = crc_remainder(msg, poly)
    frame = msg + crc
    print(f"Message: {msg}")
    print(f"CRC: {crc}")
    print(f"Frame: {frame}")
    print(f"Valid? {crc_check(frame, poly)}")

    print("\n=== Hamming Example ===")
    data = "1001"
    code = hamming_encode(data)
    print(f"Data: {data} → Code: {code}")
    decoded, ok, err = hamming_decode(code)
    print(f"Decoded: {decoded}, Valid: {ok}")
    # Introduce error
    corrupted = code[:3] + '1' + code[4:]
    decoded2, ok2, err2 = hamming_decode(corrupted)
    print(f"Corrupted: {corrupted} → Fixed? {not ok2}, Error at pos {err2}")

    print("\n=== Dijkstra Example ===")
    graph = {
        'V1': {'V2': 3, 'V3': 7, 'V4': 2, 'V7': 2},
        'V2': {'V3': 6, 'V5': 4, 'V6': 7},
        'V3': {'V6': 5},
        'V4': {'V2': 3, 'V5': 6, 'V7': 2},
        'V5': {'V3': 3, 'V6': 3, 'V8': 4},
        'V6': {'V3': 4, 'V8': 1},
        'V7': {'V2': 3, 'V5': 3, 'V8': 10},
        'V8': {}
    }
    dists = dijkstra(graph, 'V1')
    print("Shortest distances from V1:")
    for node, d in dists.items():
        print(f"  {node}: {d}")

    print("\n=== Bellman-Ford Example ===")
    edges = [
        ('A', 'B', 4), ('A', 'C', 2),
        ('B', 'C', -3), ('B', 'D', 2),
        ('C', 'D', 1)
    ]
    nodes = ['A', 'B', 'C', 'D']
    result = bellman_ford(edges, 'A', nodes)
    print("Distances (Bellman-Ford):", result)

=== CRC Example ===
Message: 101011010111
CRC: 111100
Frame: 101011010111111100
Valid? True

=== Hamming Example ===
Data: 1001 → Code: 0011001
Decoded: 1001, Valid: True
Corrupted: 0011001 → Fixed? False, Error at pos None

=== Dijkstra Example ===
Shortest distances from V1:
  V1: 0
  V2: 3
  V3: 7
  V4: 2
  V5: 5
  V6: 8
  V7: 2
  V8: 9

=== Bellman-Ford Example ===
Distances (Bellman-Ford): {'A': 0, 'B': 4, 'C': 1, 'D': 2}
